In [1]:
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset,random_split
from torchvision import datasets, transforms

import pandas as pd
import numpy as np

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

/home/viki/miniconda3/envs/WaterLevelPrediction/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


'cpu'

In [3]:
data = pd.read_csv("dummyDataTime.csv",header = None)
x=data.iloc[1:,1:6].astype('float32')
y=data.iloc[1:,-1].astype('float32')
x.iloc[:, 0] = x.iloc[:, 0] / 23
x.iloc[:, 3] = x.iloc[:, 3] / 100
x.iloc[:, 4] = x.iloc[:, 4] / 100
y = y/max(y.values)

In [4]:
x

,1,2,3,4,5
1,0.782609,1.0,1.0,0.92,0.18
2,0.217391,1.0,1.0,0.45,0.21
3,0.478261,0.0,0.0,0.03,0.48
4,0.565217,1.0,1.0,0.47,0.99
5,0.956522,1.0,0.0,0.00,0.86
...,...,...,...,...,...
99996,0.130435,1.0,1.0,0.00,0.41
99997,0.695652,0.0,1.0,0.66,0.28
99998,0.260870,1.0,0.0,0.24,0.21
99999,0.521739,1.0,1.0,0.75,0.33


In [5]:
class WaterLevelDataset(Dataset):
    def __init__(self, x, y):
        x= torch.tensor(x,dtype=torch.float32)
        y = torch.tensor(y,dtype=torch.float32)
        self.X = x
        self.Y = y

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.Y[idx]]

In [6]:
dataset = WaterLevelDataset(x.values,y.values)

In [7]:
len(dataset.X)

100000

In [8]:
dataset.X

tensor([[0.7826, 1.0000, 1.0000, 0.9200, 0.1800],
        [0.2174, 1.0000, 1.0000, 0.4500, 0.2100],
        [0.4783, 0.0000, 0.0000, 0.0300, 0.4800],
        ...,
        [0.2609, 1.0000, 0.0000, 0.2400, 0.2100],
        [0.5217, 1.0000, 1.0000, 0.7500, 0.3300],
        [0.7391, 1.0000, 0.0000, 0.0100, 0.2300]])

In [9]:
dataset.Y

tensor([0.6333, 0.1667, 0.0000,  ..., 0.0000, 0.9000, 0.0000])

In [10]:
train, test = random_split(dataset, [int(len(dataset.X)*0.8),int(len(dataset.X)*0.2)])

In [11]:
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

## Model 1
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [12]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer = nn.Linear(5, 10)
        self.layer2 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=self.layer2(X)
        X = self.activation(X)
        return X

# Instantiate the model
model = SimpleNN().to(device)



In [13]:
loss_fn = nn.MSELoss().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001)

In [14]:
for epoch in range(100):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/100], Loss: {loss.item()*100:0f}")



Epoch [1/100], Loss: 6.029233
Epoch [2/100], Loss: 8.497073
Epoch [3/100], Loss: 7.181396
Epoch [4/100], Loss: 8.893016
Epoch [5/100], Loss: 7.256571
Epoch [6/100], Loss: 9.900855
Epoch [7/100], Loss: 4.933822
Epoch [8/100], Loss: 4.733118
Epoch [9/100], Loss: 3.911271
Epoch [10/100], Loss: 2.634090
Epoch [11/100], Loss: 7.418737
Epoch [12/100], Loss: 7.731971
Epoch [13/100], Loss: 6.922608
Epoch [14/100], Loss: 8.524310
Epoch [15/100], Loss: 7.203550
Epoch [16/100], Loss: 2.778855
Epoch [17/100], Loss: 4.278284
Epoch [18/100], Loss: 3.729640
Epoch [19/100], Loss: 2.735871
Epoch [20/100], Loss: 5.777037
Epoch [21/100], Loss: 9.783365
Epoch [22/100], Loss: 6.300706
Epoch [23/100], Loss: 4.891011
Epoch [24/100], Loss: 2.452283
Epoch [25/100], Loss: 5.221869
Epoch [26/100], Loss: 2.317533
Epoch [27/100], Loss: 4.543591
Epoch [28/100], Loss: 8.054319
Epoch [29/100], Loss: 6.991445
Epoch [30/100], Loss: 8.173797
Epoch [31/100], Loss: 3.484152
Epoch [32/100], Loss: 2.027746
Epoch [33/100], L

In [15]:
model.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            test_inputs = test_inputs.to(device)
            outputs = model(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx].to(device))
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {outputs.item()*30:.2f}, Actual: {targets[idx].item()*30:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")


Predicted: 4.15, Actual: 18.00
Predicted: 1.14, Actual: 0.00
Predicted: 15.97, Actual: 13.00
Predicted: 0.94, Actual: 0.00
Predicted: 3.47, Actual: 0.00
Predicted: 11.29, Actual: 8.00
Predicted: 6.47, Actual: 29.00
Predicted: 5.70, Actual: 3.00
Predicted: 1.57, Actual: 0.00
Predicted: 3.30, Actual: 5.00
Predicted: 0.39, Actual: 0.00
Predicted: 3.82, Actual: 0.00
Predicted: 8.06, Actual: 8.00
Predicted: 6.89, Actual: 7.00
Predicted: 17.71, Actual: 27.00
Predicted: 1.64, Actual: 0.00
Average loss on the testing dataset: 5.9006
Predicted: 0.73, Actual: 0.00
Predicted: 1.13, Actual: 0.00
Predicted: 18.96, Actual: 29.00
Predicted: 10.14, Actual: 0.00
Predicted: 1.32, Actual: 0.00
Predicted: 7.03, Actual: 4.00
Predicted: 4.92, Actual: 0.00
Predicted: 15.57, Actual: 17.00
Predicted: 11.39, Actual: 28.00
Predicted: 0.70, Actual: 0.00
Predicted: 8.03, Actual: 5.00
Predicted: 3.27, Actual: 10.00
Predicted: 0.89, Actual: 0.00
Predicted: 0.56, Actual: 0.00
Predicted: 1.25, Actual: 0.00
Predicted: 

In [16]:
torch.save(model, "model/model1DataTime10000-10.h5")

## Model 2
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [12]:
class Model2NN(nn.Module):
    def __init__(self):
        super(Model2NN, self).__init__()
        self.layer = nn.Linear(5, 10)
        self.layer2 = nn.Linear(10, 10)
        self.layer3 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=torch.relu(self.layer2(X))
        X=self.layer3(X)
        X = self.activation(X)
        return X

# Instantiate the model
model1 = Model2NN().to(device)



In [13]:
loss_fn = nn.MSELoss().to(device)

optimizer = optim.SGD(model1.parameters(), lr=0.001)

In [19]:
for epoch in range(100):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model1(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/100], Loss: {loss.item()*100:0f}")



Epoch [1/100], Loss: 9.623483
Epoch [2/100], Loss: 7.116884
Epoch [3/100], Loss: 5.417408
Epoch [4/100], Loss: 4.460041
Epoch [5/100], Loss: 8.717757
Epoch [6/100], Loss: 9.123415
Epoch [7/100], Loss: 5.406176
Epoch [8/100], Loss: 4.507764
Epoch [9/100], Loss: 7.330258
Epoch [10/100], Loss: 9.722832
Epoch [11/100], Loss: 5.009720
Epoch [12/100], Loss: 3.644626
Epoch [13/100], Loss: 4.159661
Epoch [14/100], Loss: 7.911991
Epoch [15/100], Loss: 3.835241
Epoch [16/100], Loss: 7.276745
Epoch [17/100], Loss: 2.322355
Epoch [18/100], Loss: 8.349809
Epoch [19/100], Loss: 5.368762
Epoch [20/100], Loss: 3.968801
Epoch [21/100], Loss: 6.070683
Epoch [22/100], Loss: 2.216095
Epoch [23/100], Loss: 3.657331
Epoch [24/100], Loss: 2.698026
Epoch [25/100], Loss: 3.664732
Epoch [26/100], Loss: 4.291755
Epoch [27/100], Loss: 9.597460
Epoch [28/100], Loss: 3.365926
Epoch [29/100], Loss: 3.335509
Epoch [30/100], Loss: 5.398087
Epoch [31/100], Loss: 1.496371
Epoch [32/100], Loss: 5.612528
Epoch [33/100], L

In [20]:
model1.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            test_inputs = test_inputs.to(device)
            outputs = model1(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx].to(device))
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {outputs.item()*30:.2f}, Actual: {targets[idx].item()*30:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")

Predicted: 5.04, Actual: 18.00
Predicted: 0.56, Actual: 0.00
Predicted: 17.64, Actual: 13.00
Predicted: 1.76, Actual: 0.00
Predicted: 3.94, Actual: 0.00
Predicted: 10.81, Actual: 8.00
Predicted: 8.37, Actual: 29.00
Predicted: 4.66, Actual: 3.00
Predicted: 2.56, Actual: 0.00
Predicted: 3.37, Actual: 5.00
Predicted: 0.23, Actual: 0.00
Predicted: 3.84, Actual: 0.00
Predicted: 7.26, Actual: 8.00
Predicted: 8.18, Actual: 7.00
Predicted: 18.87, Actual: 27.00
Predicted: 0.77, Actual: 0.00
Average loss on the testing dataset: 5.1232
Predicted: 1.22, Actual: 0.00
Predicted: 2.04, Actual: 0.00
Predicted: 19.53, Actual: 29.00
Predicted: 8.25, Actual: 0.00
Predicted: 0.71, Actual: 0.00
Predicted: 8.42, Actual: 4.00
Predicted: 4.13, Actual: 0.00
Predicted: 14.39, Actual: 17.00
Predicted: 12.55, Actual: 28.00
Predicted: 0.40, Actual: 0.00
Predicted: 7.96, Actual: 5.00
Predicted: 2.80, Actual: 10.00
Predicted: 1.53, Actual: 0.00
Predicted: 0.19, Actual: 0.00
Predicted: 2.29, Actual: 0.00
Predicted: 1

In [21]:
torch.save(model1, "model/model2DataTime10000-10.pt")

In [27]:
test1 = torch.tensor([2/23,1,1,90/100,20/100]).to(device)
print(test1)
outputs = t(test1)
print(int(outputs.item()*30))

tensor([0.0870, 1.0000, 1.0000, 0.9000, 0.2000])
18


### Testing


In [19]:
t = torch.load("model/model2DataTime10000-10.h5",map_location=torch.device(device))

In [20]:
t.eval()

Model2NN(
  (layer): Linear(in_features=5, out_features=10, bias=True)
  (layer2): Linear(in_features=10, out_features=10, bias=True)
  (layer3): Linear(in_features=10, out_features=1, bias=True)
  (activation): Sigmoid()
)